# LLM multimodal PDF RAG

- **Author**: senkin.zhan@datarobot.com
- **Demo data**: https://s3.us-east-1.amazonaws.com/datarobot_public_datasets/ai_accelerators/pdf_demo.zip

## Summary

Since open-source PDF OCR tools often make mistakes, you can use an LLM instead. This AI accelerator introduces an approach to use an LLM as an OCR tool to extract all the text, table, and graph data from a PDF, then build a RAG and a Playround chat on DataRobot.

This notebook outlines how to:

1. Split a PDF to multiple images; one per page.
2. Extract all the text, table, and graph data from image using an LLM, then save them as markdown files. 
3. Build a vector database with the markdown files.
4. Build a Playground chat and test prompt.


## Setup

### Install and import libraries

In [1]:
!pip install google-genai pdf2image pymupdf openai anthropic -q

In [2]:
import base64
import glob
from io import BytesIO
import os
import time
import zipfile

from PIL import Image
import anthropic
import datarobot as dr
from datarobot.enums import (
    PromptType,
    VectorDatabaseChunkingMethod,
    VectorDatabaseEmbeddingModel,
)
from datarobot.models.dataset import Dataset
from datarobot.models.genai.chat import Chat
from datarobot.models.genai.chat_prompt import ChatPrompt
from datarobot.models.genai.comparison_chat import ComparisonChat
from datarobot.models.genai.comparison_prompt import ComparisonPrompt
from datarobot.models.genai.custom_model_llm_validation import CustomModelLLMValidation
from datarobot.models.genai.llm import LLMDefinition
from datarobot.models.genai.llm_blueprint import LLMBlueprint, VectorDatabaseSettings
from datarobot.models.genai.playground import Playground
from datarobot.models.genai.vector_database import (
    ChunkingParameters,
    CustomModelVectorDatabaseValidation,
    VectorDatabase,
)
import fitz
from google import genai
from google.genai import types
import numpy as np
import openai
from openai import OpenAI
import pandas as pd

### Bind variables

In [3]:
# Datarobot client
dr.Client()

# Download demo data to current directory
zip_path = "pdf_demo.zip"

# Create a folder to save pdf uncompressed from zip file
pdf_path = "pdf"
os.makedirs(pdf_path, exist_ok=True)
with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall(pdf_path)

# Create a folder to save images converted from PDF
image_path = "image"
os.makedirs(image_path, exist_ok=True)

# Create a folder to save markdown files extracted from images
markdown_path = "markdown"
os.makedirs(markdown_path, exist_ok=True)

# Generate a .zip file to create a vector database
vectordb_zip_path = "vectordb.zip"

# Image size 1000 is recommended
image_size = 1000

# Chunk parameters
chunking_method = VectorDatabaseChunkingMethod.RECURSIVE
chunk_size = 384
chunk_overlap_percentage = 50
separators = ["\n\n"]

# Playground parameters
playground_name = "multimodal_rag"
chat_name = "gpt4o"

# Chat parameters
max_completion_length = 256
temperature = 0.4
top_p = 0.9
max_documents_retrieved_per_prompt = 5
max_tokens = 384

# For previous chat prompts (history) to be included in each subsequent prompt, PromptType.ONE_TIME_PROMPT is an alternative if you don't wish
prompting_strategy = PromptType.CHAT_HISTORY_AWARE

# Max retry for creating vector database and playground
max_retry = 5

# Use jp or en version to test demo
lang = "jp"

if lang == "jp":
    # pdf_path
    pdf_path = pdf_path + "/jp"

    # prompt example
    prompt = """
        **役割設定**:
        あなたは画像からの文字認識、表の解析、および図表構造の抽出に精通したエキスパートです。

        **タスクの目的**:
        提供された画像から以下の情報を抽出し、Markdown形式で出力してください:
        1.**すべての文字情報**: 画像中に含まれる文章や段落、見出しなどを可能な限り元の構成で再現してください。
        2.**表（テーブル）**: 画像に表が含まれる場合、Markdownの表記法（| 区切り）を使って復元してください。
        3.**図表（チャート）**: グラフやチャートが含まれる場合、テキストやリスト、あるいは擬似コードなどでデータと構造をできるだけ詳しく再現してください（軸のラベル、データ点、凡例など）。

　　　　　**重要**:
　　　　　- 元の画像に含まれていない情報は絶対に付け加えないでください。余計な情報や事実にない内容の追加は避けてください。
     　　- 丁寧な挨拶や了解の返事（例：「はい、承知いたしました」など）を出力に含めないでください。

        **出力要件**:
        - Markdown形式で出力すること。
        - 元の情報を可能な限り正確に保持すること。
        - 認識できない、または不確定な部分は「不明」や「未確定」などと明示すること。
        - 文章部分は段落や箇条書きなど、わかりやすい形式で整理すること。
        - 表はMarkdownのテーブル記法を使用すること。
        - 図表については、テキストやリスト、または簡易的な擬似コードで構造を説明するなど、できるだけ細かく記述すること。
        - 全体の出力は見やすく、段階的に整理してください。
        """

    # System prompt for chat
    system_prompt = """
        質問に対して、可能な限り短く、答えてください。余計な、丁寧な言葉遣いや冗長な説明をやめてください。
        """

    # SUP_SIMCSE_JA_BASE is recommend for japanese vectordatabase
    embedding_model = VectorDatabaseEmbeddingModel.SUP_SIMCSE_JA_BASE

else:
    # pdf_path
    pdf_path = pdf_path + "/en"

    # Prompt example
    prompt = """ 
        **Role Setting**:  
        You are an expert in image text recognition, table parsing, and chart structure extraction.

        **Task Objective**:  
        Please extract the following information from the image I provide, and return the results in **Markdown** format:
        1. **All Text**: Preserve the original paragraphs, headings, or logical order as much as possible.  
        2. **Tables**: If the image contains tables, please reconstruct them using **Markdown table syntax** (using `|` as separators).  
        3. **Charts (graphs)**: If there are charts or data visualizations, try to reconstruct their structure and data. You can describe charts in text, lists, or pseudo-code (e.g., axis labels, data points, legend).

        **Important**:  
        - **Do not include any content that is not present in the original image.** Avoid adding or fabricating extra information.
        - **Do not output polite acknowledgments or confirmations** (e.g., “Yes, I understand.”).

        **Output Requirements**:  
        - The output must be in Markdown format.  
        - Retain the integrity and accuracy of the original information.  
        - If there is any text or data that is **uncertain or unrecognizable**, mark it clearly as “[Uncertain]” or “[Unrecognized]”.  
        - For text, organize it into paragraphs or bullet points.  
        - For tables, use standard Markdown table syntax.  
        - For charts, describe them in text/list form or simple pseudo-code.  
        - Keep the structure clear and organized.
        """

    # System prompt for chat
    system_prompt = """
        Answer the question as briefly as possible. Avoid unnecessary polite language and redundant explanations.
        """

    # JINA_EMBEDDING_T_EN_V1  is recommend for english vectordatabase
    embedding_model = VectorDatabaseEmbeddingModel.JINA_EMBEDDING_T_EN_V1

### Convert the PDF into images 

In [43]:
def convert_pdf_to_image(pdf_path, image_path):
    pdf_list = sorted(glob.glob(pdf_path + "/*.pdf"))
    for p in pdf_list:
        print(p)
        pdf_name = p.split(".")[0].split("/")[-1]
        doc = fitz.open(p)
        for i in range(len(doc)):
            page = doc[i]
            pix = page.get_pixmap(dpi=300)  # Adjust DPI for quality
            pix.save(f"{image_path}/{pdf_name}_{i+1}.jpeg")  # Save as PNG


convert_pdf_to_image(pdf_path, image_path)

pdf/jp/ドラえもん.pdf
pdf/jp/ポケットモンスター.pdf


### Check the LLM OCR processing of one image

In [4]:
def llm_generate_content(llm, model, api_key, api_version, endpoint, prompt, img_path):
    if llm == "google":
        image = Image.open(img_path)
        client = genai.Client(api_key=api_key)
        response = client.models.generate_content(model=model, contents=[prompt, image])
        res = response.text

    if llm == "anthropic":
        with open(img_path, "rb") as f:
            bytes = f.read()
            image = base64.standard_b64encode(bytes).decode("utf-8")
        client = anthropic.Anthropic(api_key=api_key)
        response = client.messages.create(
            max_tokens=4096,
            model=model,
            messages=[
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "image",
                            "source": {"type": "base64", "media_type": "image/jpeg", "data": image},
                        },
                        {"type": "text", "text": prompt},
                    ],
                }
            ],
        )

        res = response.content[0].text

    if llm == "openai":
        with Image.open(img_path) as img:
            buffered = BytesIO()
            img.save(buffered, format="JPEG")
            image = base64.b64encode(buffered.getvalue()).decode("utf-8")
        client = OpenAI(api_key=api_key)
        response = client.chat.completions.create(
            model=model,
            messages=[
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": prompt},
                        {
                            "type": "image_url",
                            "image_url": {"url": f"data:image/jpeg;base64,{image}"},
                        },
                    ],
                }
            ],
            max_tokens=4096,
        )

        res = response.choices[0].message.content

    if llm == "azure":
        openai.api_type = llm
        openai.azure_endpoint = endpoint
        openai.api_version = api_version
        openai.api_key = api_key
        with Image.open(img_path) as img:
            buffered = BytesIO()
            img.save(buffered, format="JPEG")
            image = base64.b64encode(buffered.getvalue()).decode("utf-8")
        response = openai.chat.completions.create(
            model=model,
            messages=[
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": prompt},
                        {
                            "type": "image_url",
                            "image_url": {"url": f"data:image/jpeg;base64,{image}"},
                        },
                    ],
                }
            ],
            max_tokens=4096,
        )

        res = response.choices[0].message.content

    return res


# select llm service provider from (google | azure | openai | anthropic), (gemini-2.0-flash | claude-3-7-sonnet-20250219 | gpt-4o) are recommended for image ocr task

## google gemini-2.0-flash
# llm = 'google'
# model = 'gemini-2.0-flash'
# api_key = 'your-api-key'
# api_version = '' # keep empty
# endpoint = ''  # keep empty

## anthropic claude 3.7
# llm = 'anthropic'
# model = "claude-3-7-sonnet-20250219"
# api_key = 'your-api-key'
# api_version = '' # keep empty
# endpoint = ''  # keep empty

## openai gpt-4o
# llm = 'openai'
# model = 'gpt-4o'
# api_key = 'your-api-key'
# api_version = '' # keep empty
# endpoint = ''  # keep empty


## azure gpt-4o
llm = "azure"
model = "gpt-4o"
api_key = "your-api-key"
api_version = "2024-10-21"  # Use the correct version for your deployment
endpoint = "https://your-organization.openai.azure.com/"  # Use the correct azure endpoint for your deployment

img_path = sorted(glob.glob(image_path + "/*.jpeg"))[0]
llm_generate_content(llm, model, api_key, api_version, endpoint, prompt, img_path)

'```markdown\n# ドラえもん\n出典: フリー百科事典『ウィキペディア (Wikipedia)』\n\n藤子不二雄（連載） > 藤子・F・不二雄（著作） > ドラえもん\n\n『ドラえもん』は、藤子・F・不二雄[注釈 1]による日本のSF児童ギャグ漫画である。1969年から主に児童向け雑誌で「藤子不二雄」名義で連載が開始[1][2]された。開始当初から藤本弘単独作品。1989年以降は「藤子・F・不二雄」名義となった。日本では国民的な知名度があり、海外でも東アジアや東南アジアを中心に高い人気を誇る[4]。2012年9月には藤子・F・不二雄大全集『ドラえもん』全20巻が完結し、藤本によって描かれた1300以上のドラえもん漫画作品のほぼ全話が単行本に収録された。\n\n## 作品の概要\n22世紀の未来からやってきたネコ型ロボット・ドラえもんと、勉強もスポーツも苦手な小学生・野比のび太が繰り広げる日常生活を描いた作品である。基本的には一話完結型の連載漫画だが、連続ストーリー型となって日常を離れた冒険をする「大長編」シリーズもある。話完結の基本的なプロットは、ドラえもんがポケットから出す多種多様なひみつ道具（現代の技術では再現も実現も不可能な機能を持つ）でのび太（或いは他の場合もある）の身にふりかかる災難を解決しようとするが、道具を不適切に使った結果、しっぺ返しを受けるといったものが多い。\n\n## あらすじ\nのび太がふだんのんびりと過ごしていると、突然、どこからともなく彼の未来を告げる声が聞こえ、机の引き出しからドラえもんと、のび太の孫の孫のセワシが現れた。セワシ曰く、のび太は社会に出た後も...\n\n---\n\n![ドラえもんの主要キャラクターの像 (高岡おとぎの森公園内「ドラえもんの空き地」より)]()\n\n## テーブル\n| ジャンル | 児童漫画、少年漫画、SF漫画、ギャグ漫画 |\n| --- | --- |\n| 形式 | 漫画 |\n| 作者 | 藤子・F・不二雄[注釈 1] |\n| 出版社 | 小学館 |\n| その他の出版社 | 中央公論社（FFランド） |\n| 掲載誌 | 小学館の学習雑誌、コロコロコミック、てれびくん 他 |\n| レーベル | てんとう虫コミックス 他 |\n| 発表期間 | 1969年 - 1997年

### Extract markdown text from all of the images

In [5]:
%%time


def extract_markdown_from_image(
    image_path, markdown_path, vectordb_zip_path, llm, model, api_key, api_version, endpoint, prompt
):
    images = sorted(glob.glob(image_path + "/*.jpeg"))
    df = pd.DataFrame()
    for m in images:
        print("processing ", m, "......")
        text_page = llm_generate_content(llm, model, api_key, api_version, endpoint, prompt, m)
        tmp = pd.DataFrame({"text_page": [text_page]})
        tmp["source"] = m
        tmp["page"] = tmp["source"].apply(lambda x: x.split(".")[0].split("_")[-1]).astype(int)
        tmp["pdf"] = tmp["source"].apply(lambda x: x.split("/")[-1].split("_")[0])
        df = pd.concat([df, tmp])

    df = df.sort_values(["pdf", "page"]).reset_index(drop=True)
    df_source = df.groupby(["pdf"])["text_page"].agg(list).reset_index()
    df_source["text"] = df_source["text_page"].apply(lambda x: "\n\n".join(x))
    df_source = df_source.drop(["text_page"], axis=1).reset_index(drop=True)

    for i in range(len(df_source)):
        filename = df_source.iloc[i]["pdf"]
        text = df_source.iloc[i]["text"]
        with open(markdown_path + "/" + filename + ".md", "w", encoding="utf-8_sig") as text_file:
            text_file.write(text)

    with zipfile.ZipFile(vectordb_zip_path, "w", zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(markdown_path):
            for file in files:
                filepath = os.path.join(root, file)
                arcname = os.path.relpath(filepath, start=markdown_path)
                zipf.write(filepath, arcname)


extract_markdown_from_image(
    image_path, markdown_path, vectordb_zip_path, llm, model, api_key, api_version, endpoint, prompt
)

processing  image/ドラえもん_1.jpeg ......
processing  image/ドラえもん_10.jpeg ......
processing  image/ドラえもん_11.jpeg ......
processing  image/ドラえもん_12.jpeg ......
processing  image/ドラえもん_13.jpeg ......
processing  image/ドラえもん_14.jpeg ......
processing  image/ドラえもん_15.jpeg ......
processing  image/ドラえもん_16.jpeg ......
processing  image/ドラえもん_17.jpeg ......
processing  image/ドラえもん_18.jpeg ......
processing  image/ドラえもん_19.jpeg ......
processing  image/ドラえもん_2.jpeg ......
processing  image/ドラえもん_20.jpeg ......
processing  image/ドラえもん_21.jpeg ......
processing  image/ドラえもん_22.jpeg ......
processing  image/ドラえもん_23.jpeg ......
processing  image/ドラえもん_24.jpeg ......
processing  image/ドラえもん_25.jpeg ......
processing  image/ドラえもん_26.jpeg ......
processing  image/ドラえもん_27.jpeg ......
processing  image/ドラえもん_28.jpeg ......
processing  image/ドラえもん_29.jpeg ......
processing  image/ドラえもん_3.jpeg ......
processing  image/ドラえもん_30.jpeg ......
processing  image/ドラえもん_31.jpeg ......
processing  image/ドラえもん_32.j

### Build a Use Case and vector database with the extracted markdown files

In [15]:
%%time
def create_vectordb(file):
    use_case = dr.UseCase.create(file.split(".")[0])
    dataset = dr.Dataset.create_from_file(file)
    use_case.add(entity=dataset)

    chunking_parameters = ChunkingParameters(
        embedding_model=embedding_model,
        chunking_method=chunking_method,
        chunk_size=chunk_size,
        chunk_overlap_percentage=chunk_overlap_percentage,
        separators=separators,
    )
    vdb = VectorDatabase.create(dataset.id, chunking_parameters, use_case)

    for n in range(max_retry):
        time.sleep(60)
        try:
            vdb = VectorDatabase.get(vdb.id)
            assert vdb.execution_status == "COMPLETED"
        except:
            continue
        else:
            break

    return use_case, vdb


use_case, vdb = create_vectordb(vectordb_zip_path)

CPU times: user 649 ms, sys: 7.74 ms, total: 656 ms
Wall time: 5min 21s


### Build an LLM playground 

In [16]:
playground = Playground.create(name=playground_name, use_case=use_case)
llms = LLMDefinition.list(use_case=use_case, as_dict=False)

In [17]:
llms

[LLMDefinition(id=azure-openai-gpt-3.5-turbo, name=Azure OpenAI GPT-3.5 Turbo),
 LLMDefinition(id=azure-openai-gpt-3.5-turbo-16k, name=Azure OpenAI GPT-3.5 Turbo 16k),
 LLMDefinition(id=azure-openai-gpt-4, name=Azure OpenAI GPT-4),
 LLMDefinition(id=azure-openai-gpt-4-32k, name=Azure OpenAI GPT-4 32k),
 LLMDefinition(id=azure-openai-gpt-4-turbo, name=Azure OpenAI GPT-4 Turbo),
 LLMDefinition(id=azure-openai-gpt-4-o, name=Azure OpenAI GPT-4o),
 LLMDefinition(id=azure-openai-gpt-4-o-mini, name=Azure OpenAI GPT-4o Mini),
 LLMDefinition(id=amazon-titan, name=Amazon Titan),
 LLMDefinition(id=anthropic-claude-2, name=Anthropic Claude 2.1),
 LLMDefinition(id=anthropic-claude-3-haiku, name=Anthropic Claude 3 Haiku),
 LLMDefinition(id=anthropic-claude-3-sonnet, name=Anthropic Claude 3 Sonnet),
 LLMDefinition(id=anthropic-claude-3-opus, name=Anthropic Claude 3 Opus),
 LLMDefinition(id=google-bison, name=Google Bison),
 LLMDefinition(id=google-gemini-1.5-flash, name=Google Gemini 1.5 Flash),
 LLM

In [18]:
# use azure-openai-gpt-4-o-mini as chat llm
gpt = llms[6]
gpt

LLMDefinition(id=azure-openai-gpt-4-o-mini, name=Azure OpenAI GPT-4o Mini)

### Build an LLM blueprint

In [19]:
llm_settings = {
    "system_prompt": (system_prompt),
    "max_completion_length": max_completion_length,
    "temperature": temperature,
    "top_p": top_p,
}

prompting_strategy = prompting_strategy

vector_database_settings = VectorDatabaseSettings(
    max_documents_retrieved_per_prompt=max_documents_retrieved_per_prompt,
    max_tokens=max_tokens,
)

llm_blueprint = LLMBlueprint.create(
    playground=playground,
    name="GPT",
    llm=gpt,
    prompt_type=prompting_strategy,
    llm_settings=llm_settings,
    vector_database=vdb,
    vector_database_settings=vector_database_settings,
)

## Build a chat

In [20]:
chat = Chat.create(name=chat_name, llm_blueprint=llm_blueprint)

## Test the prompt

In [21]:
prompt = ChatPrompt.create(
    chat=chat,
    text="ドラえもん最初テレビ放送はいつですか？",
    wait_for_completion=True,
)
print(prompt.result_text)

1973年です。


In [22]:
prompt = ChatPrompt.create(
    chat=chat,
    text="2024年3月末時点ポケモン関連ゲーム累計出荷数は？",
    wait_for_completion=True,
)
print(prompt.result_text)

4億8000万本以上です。
